In [0]:
# ============================================================
# Lectura de la tabla Silver particionada para crear la capa Gold
# ============================================================

# Usamos spark.read.table para cargar la tabla Delta 'df_techmart_silver_partitioned'
# que corresponde a la capa Silver ya limpia y particionada por mes.
# Esta lectura nos entrega un DataFrame Spark llamado 'df_TechMart_goldstage',
# el cual servirá como base para aplicar las transformaciones y agregaciones
# necesarias en la capa Gold (ej. métricas, tendencias, top productos).

df_TechMart_goldstage = spark.read.table("workspace.default.df_techmart_silver_partitioned")

In [0]:
# ============================================================
# Cálculo de métricas globales en la capa Gold
# ============================================================

import pyspark.sql.functions as F

# 1. Agregación de métricas principales
# - ventas_totales: suma de la columna 'total'
# - num_transacciones: conteo de registros por id_venta
# - ticket_promedio: promedio del monto por transacción
ventas_totales = (
    df_TechMart_goldstage
    .agg(
        F.sum("total").alias("ventas_totales"),
        F.count("id_venta").alias("num_transacciones"),
        (F.sum("total") / F.count("id_venta")).alias("ticket_promedio")
    )
    # 2. Formateo de métricas monetarias
    # Se convierten a string con símbolo $ y dos decimales
    .withColumn("ventas_totales", F.format_string("$%.2f", F.col("ventas_totales")))
    .withColumn("ticket_promedio", F.format_string("$%.2f", F.col("ticket_promedio")))
)

# 3. Escritura en la capa Gold
# Guardamos la tabla en formato Delta Lake con modo 'append'
# para permitir cargas incrementales sin sobrescribir datos previos.
ventas_totales.write.format("delta").mode("append").saveAsTable("df_techmart_gold_ventas_totales")


In [0]:
# ============================================================
# Cálculo de métricas por sucursal en la capa Gold
# ============================================================

ventas_por_sucursal = (
    df_TechMart_goldstage
    # 1. Agrupamos por la columna 'sucursal'
    .groupBy("sucursal")
    # 2. Calculamos métricas principales por cada sucursal:
    # - ventas_totales: suma de la columna 'total'
    # - num_transacciones: conteo de registros por id_venta
    # - ticket_promedio: promedio del monto por transacción
    .agg(
        F.sum("total").alias("ventas_totales"),
        F.count("id_venta").alias("num_transacciones"),
        (F.sum("total") / F.count("id_venta")).alias("ticket_promedio")
    )
    # 3. Formateamos las métricas monetarias
    # Se convierten a string con símbolo $ y dos decimales
    .withColumn("ventas_totales", F.format_string("$%.2f", F.col("ventas_totales")))
    .withColumn("ticket_promedio", F.format_string("$%.2f", F.col("ticket_promedio")))
)

# 4. Escritura en la capa Gold
# Guardamos la tabla en formato Delta Lake con modo 'append'
# para permitir cargas incrementales sin sobrescribir datos previos.
ventas_por_sucursal.write.format("delta").mode("append").saveAsTable("df_techmart_gold_ventas_por_sucursal")


In [0]:
# ============================================================
# Cálculo del Top 10 productos más vendidos en la capa Gold
# ============================================================

top_productos = (
    df_TechMart_goldstage
    # 1. Agrupamos por la columna 'producto'
    .groupBy("producto")
    # 2. Calculamos la cantidad total vendida por producto
    .agg(F.sum("cantidad").alias("cantidad_vendida"))
    # 3. Ordenamos los productos de mayor a menor según la cantidad vendida
    .orderBy(F.desc("cantidad_vendida"))
    # 4. Limitamos el resultado a los 10 primeros productos
    .limit(10)
)

# 5. Escritura en la capa Gold
# Guardamos la tabla en formato Delta Lake con modo 'append'
# para permitir cargas incrementales sin sobrescribir datos previos.
top_productos.write.format("delta").mode("append").saveAsTable("df_techmart_gold_top_productos")



In [0]:
# ============================================================
# Cálculo de métricas por categoría en la capa Gold
# ============================================================

ventas_por_categoria = (
    df_TechMart_goldstage
    # 1. Agrupamos por la columna 'categoria'
    .groupBy("categoria")
    # 2. Calculamos métricas principales por cada categoría:
    # - ventas_totales: suma de la columna 'total'
    # - productos_vendidos: suma de la columna 'cantidad'
    .agg(
        F.sum("total").alias("ventas_totales"),
        F.sum("cantidad").alias("productos_vendidos")
    )
    # 3. Formateamos las métricas monetarias
    # Se convierten a string con símbolo $ y dos decimales
    .withColumn("ventas_totales", F.format_string("$%.2f", F.col("ventas_totales")))
)

# 4. Escritura en la capa Gold
# Guardamos la tabla en formato Delta Lake con modo 'overwrite'
# para reemplazar completamente los datos previos con los nuevos cálculos.
ventas_por_categoria.write.format("delta").mode("overwrite").saveAsTable("df_techmart_gold_ventas_por_categoria")


In [0]:
# ============================================================
# Cálculo de la tendencia mensual de ventas en la capa Gold
# ============================================================

tendencia_mensual = (
    df_TechMart_goldstage
    # 1. Agrupamos por año y mes
    .groupBy("año", "mes")
    # 2. Calculamos métricas principales por cada periodo:
    # - ventas_totales: suma de la columna 'total'
    # - num_transacciones: conteo de registros por id_venta
    # - productos_vendidos: suma de la columna 'cantidad'
    .agg(
        F.sum("total").alias("ventas_totales"),
        F.count("id_venta").alias("num_transacciones"),
        F.sum("cantidad").alias("productos_vendidos")
    )
    # 3. Ordenamos cronológicamente por año y mes
    .orderBy("año", "mes")
    # 4. Formateamos las métricas monetarias
    # Se convierten a string con símbolo $ y dos decimales
    .withColumn("ventas_totales", F.format_string("$%.2f", F.col("ventas_totales")))
)

# 5. Escritura en la capa Gold
# Guardamos la tabla en formato Delta Lake con modo 'overwrite'
# para reemplazar completamente los datos previos con los nuevos cálculos.
tendencia_mensual.write.format("delta").mode("overwrite").saveAsTable("df_techmart_gold_tendencia_mensual")
